## Algoritmo de recomendacion basado en contenido

> En caso se utilizó la técnica TF-IDF (Term Frequency-Inverse Document Frequency) es una técnica que nos ayuda con el reconocimiento del lenguaje natural, su propósito es evaluar la importancia de una palabra o termino especifico en un texto en relación con su frecuencia en un conjunto.  En este caso calculamos las similitudes entre las descripciones de cada libro, también se calculo la similitud de los cosenos para determinar la similitud de las descripciones entre las descripciones de los libros 

#### Carga y tratamiento de los datos

Se importan los datos usando pandas y se muestran las 10 primeras filas

In [2]:
import pandas as pd

data = pd.read_csv("books_data.csv")   
data.head(5)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


Se hace una descripcion de los datos donde se describe el tipo de dato y la cantidad de datos no-nulos por columnas

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


Cantidad de datos nulos por columnas

In [4]:
data.isnull().sum()

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64

Porcentaje de valores nulos por columnas

In [5]:
data.isnull().mean() * 100

Title             0.000471
description      32.222557
authors          14.789270
image            24.516958
previewLink      11.222011
publisher        35.727199
publishedDate    11.913617
infoLink         11.222011
categories       19.396527
ratingsCount     76.576712
dtype: float64

Para nuestro modelo vamos a usar el titulo y la descripcion del libro por lo que las otras variables las podemos eliminar 

In [6]:
data = data.loc[:, ['Title','description']]

Ahora el 32% de las filas de la columna descripción esta vacíos, en este caso los vamos a rellenar con una cadena de texto vacía para conservar la integridad de los datos y tener una buena compatibilidad ya que al llenar con texto vacío los valores nulos no se interrumpe con el flujo de trabajo del procesamiento de texto, dado que el texto vacío no tiene contenido su representación TF-IDF consistirá en vectores de ceros En otras palabras, los libros sin descripción serán menos similares entre sí y con otros libros que tengan descripciones, lo cual es lógico y deseable.

In [7]:
data['description'] = data['description'].fillna('')

Convertimos el texto en minúscula para evitar errores de tipado


In [8]:
data['description'] = data['description'].str.lower()

Importamos las biblioecas que vamos a utilizar 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Aplicamos la técnica TF-IDF

In [10]:
data = data.sample(n=30000)

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['description'])

Calculamos la similitud de similitud coseno entre descripciones

> Nota: se tubo que coger una muestra de 20000 datos porque estaba generando error de run time al momento de hacer el calculo

In [12]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

La siguiente funcion nos ayuda a escoges los tres libros mas parecidos al que se ingreso

In [13]:
def obtener_recomendacion(title, cosine_sim=cosine_sim, data=data, top_n=3):

    if title not in data['Title'].values:
        print(f"El título '{title}' no se encuentra en el conjunto de datos.")
        return []
    
    idx = data[data['Title'] == title].index[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_scores = sim_scores[1:top_n+1]
    
    book_indices = [i[0] for i in top_scores]
    
    return data['Title'].iloc[book_indices]


In [16]:
print(obtener_recomendacion("Dr. Seuss: American Icon"))

15701                           Healing the Past God's Way
54830                                   The Pain of Loving
82239    Broken Open: How Difficult Times Can Help Us Grow
Name: Title, dtype: object
